In [8]:
from helpers.imagenet_classes import IMAGENET2012_CLASS_INDEXES
import os

def imagenet_samples(split: str):
    for root, dirs, files in os.walk(f"data/{split}", topdown=False):
        for fname in files:
            if not fname.endswith(".JPEG"):
                continue
            fpath = os.path.join(root, fname)
            with open(fpath, "rb") as stream:
                binary_data = stream.read()
            
            _, synset_id = os.path.splitext(fname)[0].rsplit("_", 1)
            label = IMAGENET2012_CLASS_INDEXES[synset_id]
            
            sample = {
                "__key__": os.path.splitext(fname)[0],
                "jpg": binary_data,
                "cls": label
            }
            yield sample

print(next(imagenet_samples("val"))["cls"])

91


In [14]:
import webdataset as wds

def write_out(split: str):
    os.makedirs(f"data_out/{split}", exist_ok=True)
    with wds.ShardWriter(f"data_out/{split}/%06d.tar", maxcount=10000) as sink:
        for sample in imagenet_samples(split):
            sink.write(sample)

write_out("val")

# writing data_out/val/000000.tar 0 0.0 GB 0
# writing data_out/val/000001.tar 10000 1.3 GB 10000
# writing data_out/val/000002.tar 10000 1.3 GB 20000
# writing data_out/val/000003.tar 10000 1.3 GB 30000
# writing data_out/val/000004.tar 10000 1.3 GB 40000
